In [26]:
import logging
import math
import numpy as np
import pandas as pd
import os
import random
import spacy
import sys
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.nn.utils import clip_grad_norm_

from glob import glob
from tqdm import tqdm, tqdm_notebook

In [4]:
sys.path.append('../../NLPer')

In [5]:
from nlper.utils.lang_utils import Token
from nlper.trainer.data_loader import DataLoader
from nlper.utils.lang_utils import VocabConfig

from nlper.utils.torch_utils import get_device
from nlper.utils.torch_utils import AVAILABLE_GPU

from nlper.utils.train_utils import calculate_rouge
from nlper.utils.train_utils import draw_attention_matrix

In [27]:
logging.basicConfig(
    format=f"%(asctime)s [%(levelname)s] | %(name)s | %(funcName)s: %(message)s",
    level=logging.INFO,
    datefmt='%I:%M:%S',
)

## Load data

In [6]:
glob('../../NLPer/resources/output/trimmed_all_data/*')

['../../NLPer/resources/output/trimmed_all_data/val.csv',
 '../../NLPer/resources/output/trimmed_all_data/test.csv',
 '../../NLPer/resources/output/trimmed_all_data/train.csv']

In [7]:
pd.read_csv('../../NLPer/resources/output/trimmed_all_data/val.csv', index_col=0).head()

,summary
text,
niezbyt liczny zgromadzenie być bodaj pierwszy demonstracja protest który odbyć się w Rosja po ogłoszenie w czwartka decyzja o podniesienie wieko emerytalny . wiece organizować m.in. partia emeryt a wziąć w on udział aktywista radykalny front lewica jak i zwolennik jeden z przywódca opozycja antykremlowskiej Aleksiej nawalnego . uczestnik protest przynieść na wiece plakata głosić nie dla podniesienie wieko emerytalny i nie chcieć umrzeć w praca . żądać dymisja premier dmitrija miedwiediewa . pojawić się także hasło putin rosja <num> <num> nawiązywać do wynik czwartkowy mecz między reprezentacja Rosja i Arabia saudyjski . mecz ten zainaugurować odbywać się w Rosja mistrzostwo świat w piłka nożny . ten sam dzień rząd ogłosić plan stopniowy podwyższania od przyszły rok wieko emerytalny . dziennik wiedomosti podać w sobota powoływać się na źródło na kreml że władza Rosja niepokoić się perspektywa protest społeczny . decyzja o podniesienie wieko emerytalny według ekonomista nieunikniony być długo odkładać bowiem być bardzo popularny w społeczeństwo . obecny wieko emerytalny w Rosja to <num> rok dla mężczyzna i <num> dla kobieta . rząd chcieć podnieść ten próg do <num> rok dla mężczyzna i <num> dla kobieta przy co proces ten mieć być rozłożyć na bliski <num> <num> rok . petycja przeciw podniesienie wieko emerytalny skierować do prezydent Władimir Putin premier dmitrija miedwiediewa i władza oba izba parlament Rosja pojawić się w internet na strona change.org . podpisać on do piątek <num> tys. osoba . autor petycja przywoływać statystyka mówić o to iż w <num> region federacja rosyjski średnia prognozować długość życie być niski niż <num> rok . według dane rosyjski ministerstwo zdrowie ogół przeciętny długość życie przekroczyć w Rosja <num> rok . przy co dla kobieta wskaźnik ten wynosić ponad <num> rok a dla mężczyzna <num> rok . z Moskwa anna wróbel papa awl mobr mrr,rosja w Nowosybirsk odbyć się wiece przeciwko ...
resovia <num> lokat w środa niby tylko zremisować ale warto zauważyć że wcześnie pelikan u siebie wszystek rywal odprawiać z kwitek . w ekipa z Wolbrom <num> grać kilka groźny zawodnik jak rak dudziński ale bardzo znany być trener . Antonie szymanowski to obrońca słynny drużyna Kazimierz Górski . coach przebój znany być z to że lubić sobie ponarzekać . być nieźle gdyby w sobota mieć ku to rzeczywisty powód . przebój nie być tak mocny jak kilka rok temu jednak mieć parę rutyniarz . bardzo chcieć się zrehabilitować za mecz z rucho mówić hajda . mecz w Brzesko będzie mieć wiele dodatkowy smaczek . wielki to fakt iż Czesława palik coach stal niedawno trenować okocimskiego wprowadzić on do zreformować ii liga i mieć do dyspozycja Ireneusz Gryboś obecnie gracz stal . w skład piwosz grać kilka były stalowiec Ogara Popiel matras szósty ekipa tabela dowodzić Krzysztofa łętocha były zawodnik stal swój czas wymieniać jako kandydat do on trenowania . łętiemu i on piłkarz ostatnio iść szczególnie na wyjazd . wiedzieć że potrafić grać w piłka widziałem jak pokonać w puchar resovię podkreślać palik . trochę mój serce w Brzesko zostać . trenowałem niemal wszystek obecny gracz ten drużyna . ten wiedza się przydać . w obóz biało niebieski panować średni nastrój . kontuzja załapać Wojciechy krauze naciągnięcie mięsień dwugłowy zbić podbicie mieć serges kiema . jeśli dodać do to że ibrahim sunday dopiero odbudowywać forma gryboś wracać po kontuzja a udoudo ciągle pudłować szansa stal nie wyglądać za dobrze . wesoło nie być ale zawsze starać się myśleć pozytywnie i zarażać ten piłkarz . będziemy walczyć o punkt zakończyć trener rzeszowianin . resovia Barany kontuzja stal solecki cieślik federkiewicz krauze kiema . kontuzja .,resovia zagrać z przebój wolbrom Stala w Brzes...
być gnijąca panna młody i być eksplodującysamsung galaxy note <num> samsung wstrzymać produkcja urządzenie i namawiać on posiadacz do dokonywania zwrot . a co w ten sytuacja z pozostały akcesorium . przeważnie wydatek związane z zakup smartfon nie kończyć si

In [8]:
nlp = spacy.load('pl_spacy_model', disable=['ner', 'parser'])

In [9]:
config = {
    'train_test_val_dir': '../../NLPer/resources/output/trimmed_all_data/',
    'model_output_path': '../../NLPer/resources/model_files_notebook/',
    'vocab_output_path': '../../NLPer/resources/vocab_files_notebook/',
    'model_name': 'seq2seq_with_att_pl_base',
    'min_frequency_of_words_in_vocab': 10,
    'dataframes_field_names': ['text', 'summary'],
    'batch_size': 16,
    'hidden_size': 256,
    'embed_size': 128,
    'epochs': 100,
    'learning_rate': 0.01,
    'grad_clip': 10.0,
    'scheduler_step_size': 5000,
    'scheduler_gamma': 0.75,
    'save_model_after_epoch': True,
}

## Get dataset

In [10]:
data_iterators, TEXT, SUMMARY = DataLoader(config=config).load()

11:23:06 [INFO] | LangUtils | set_language_model: Language model using SpaCy `pl_spacy_model`
11:23:34 [INFO] | DataLoader | load: Length of vocabulary 68725


## Get Vocabulary

In [11]:
vocab_config = VocabConfig()
vocab_config.set_vocab_from_field(TEXT)

In [12]:
train_iter, valid_iter, test_iter = data_iterators

In [13]:
batch = next(iter(train_iter))

#### Random text

In [14]:
vocab_config.text_from_indices(batch.text[0].transpose(0, 1)[-1])

'<sos> na złoty oczywiście oddziaływać będą taki wydarzenie jak decyzja główny bank światowy przed wszystko rezerwa federalny mówić Sławomira dębowski główny analityka globtrex.com . natomiast co do polski rzeczywistość to oczywiście znaczenie będą mieść wybór parlamentarny który odbyć się na jesień . oczywiście pis mieć duży przewaga nad platforma obywatelski i wyglądać na to że będziemy mieć na jesień przejęcie władza i to się inwestor obawiać . myślić że poziom <num> na euro złoty czy poziom <num> na dolar do złoty móc być w ciąg ten kilka miesiąc do osiągnięcie . jak mówić z analiza techniczny wynikać że w długi perspektywa dolar móc się znacząco wzmocnić w relacja do euro . można oczekiwać nawet kurs około <num> który być tegoroczny minimum . a to automatycznie będzie sprzyjać <unk> się polski waluta . spadek móc też dotknąć giełda . już wydarzenie po wybór prezydencki pokazać że polski rynka być wrażliwy na ten typ zmiana . w kwestia korelacja pomiędzy polski rynek a rynka zagran

#### Random summary

In [15]:
vocab_config.text_from_indices(batch.summary[0].transpose(0, 1)[-1])

'<sos> Złoto móc osłabić się do główny waluta o <num> grosze . przez wybór parlamentarny na walutowy rynek w Polska inwestor szykować się do wybór parlamentarny . z analiza ekonomista wynikać że obawa przed rząd prawo i sprawiedliwość móc znacząco osłabić polski waluta . jesień móc płacić za dolar niemal <num> złoty a za euro nawet <num> złoty . przysłużyć się to także sytuacja międzynarodowy a analiza techniczny potwierdzać ten poziom . <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> '

## Seq2Seq model

In [16]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, n_layers=1, dropout=0.1):
        super(EncoderRNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.embedding = nn.Embedding(
            input_size, embedding_size, padding_idx=1).to(get_device())
        self.gru = nn.GRU(embedding_size, hidden_size, n_layers,
                          dropout=dropout, bidirectional=True).to(get_device())

    def forward(self, sequence, hidden=None):
        embedding_output = self.embedding(sequence)  # max_text_len x batch_size x embedding_size
        encoder_outputs, hidden = self.gru(embedding_output, hidden)
        # hidden: bidirectional x batch_size x hidden_size
        # output: max_text_len x batch_size x bidirectional * hidden_size
        encoder_outputs = encoder_outputs[:, :, :self.hidden_size] + encoder_outputs[:, :, self.hidden_size:]
        # output: max_text_len x batch_size x hidden_size
        return encoder_outputs, hidden

In [17]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.hidden_size = hidden_size
        self.attention = nn.Linear(hidden_size * 2, hidden_size).to(get_device())
        self.v = nn.Parameter(torch.rand(hidden_size)).to(get_device())
        stdv = 1. / math.sqrt(self.v.size(0))
        self.v.data.uniform_(-stdv, stdv)

    def forward(self, hidden, encoder_outputs):
        h = hidden.transpose(0, 1).repeat(1, encoder_outputs.size(0), 1)
        encoder_outputs = encoder_outputs.transpose(0, 1)
        attn_energies = self.score(h, encoder_outputs)  # batch_size x t x hidden
        return F.softmax(attn_energies, dim=1).unsqueeze(1)  # batch_size x t

    def score(self, hidden, encoder_outputs):
        # batch_size x t x 2*hidden -> batch_size x t x hidden
        energy = torch.tanh(self.attention(torch.cat([hidden, encoder_outputs], 2)))
        energy = energy.transpose(1, 2)  # batch_size x t x 2*hidden -> batch_size x t x hidden
        v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)  # batch_size x 1 x hidden
        energy = torch.bmm(v, energy)  # batch_size x 1 x t
        return energy.squeeze(1)  # batch_size x t

In [18]:
class DecoderRNN(nn.Module):
    def __init__(self, embedding_size, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(DecoderRNN, self).__init__()
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(
            output_size, embedding_size, padding_idx=1).to(get_device())
        self.dropout = nn.Dropout(dropout, inplace=True).to(get_device())
        self.attention = BahdanauAttention(hidden_size).to(get_device())
        self.gru = nn.GRU(hidden_size + embedding_size, hidden_size, n_layers, dropout=dropout).to(get_device())
        self.classifier = nn.Linear(hidden_size * 2, output_size).to(get_device())

    def forward(self, sequence, hidden, encoder_outputs):
        # Get the embedding of the current input word (last output word)
        embedding_output = self.embedding(sequence).unsqueeze(0)  # 1 x batch_size x n
        embedding_output = self.dropout(embedding_output)
        # Calculate attention weights and apply to encoder outputs
        attention_weights = self.attention(hidden, encoder_outputs)
        context = attention_weights.bmm(encoder_outputs.transpose(0, 1))  # batch_size x 1 x n
        context = context.transpose(0, 1)  # (1,B,N)
        # Combine embedded input word and attended context, run through RNN
        decoder_input = torch.cat([embedding_output, context], 2)
        decoder_output, hidden = self.gru(decoder_input, hidden)
        decoder_output = decoder_output.squeeze(0)  # (1,B,N) -> (B,N)
        decoder_output = self.classifier(torch.cat([decoder_output, context.squeeze(0)], 1))
        decoder_output = F.log_softmax(decoder_output, dim=1)
        return decoder_output, hidden, attention_weights

In [19]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, text, summary, teacher_forcing_ratio=0.5):
        batch_size = text.size(1)
        max_len = summary.size(0)
        vocab_size = self.decoder.output_size

        encoder_output, hidden = self.encoder(text)
        hidden = hidden[:self.decoder.n_layers]
        output = summary.data[0, :]  # sos

        outputs = torch.FloatTensor(max_len, batch_size, vocab_size).fill_(0).to(get_device())
        for t in range(1, max_len):
            output, hidden, attention_weights = self.decoder(
                output, hidden, encoder_output)
            outputs[t] = output
            is_teacher = random.random() < teacher_forcing_ratio
            top_first = output.data.max(1)[1]
            output = summary.data[t] if is_teacher else top_first
        return outputs

In [20]:
print("[!] Instantiating model...")
encoder = EncoderRNN(
    input_size=config['text_size'],
    embedding_size=config['embed_size'],
    hidden_size=config['hidden_size'],
    n_layers=2,
    dropout=0.5,
)
decoder = DecoderRNN(
    embedding_size=config['embed_size'],
    hidden_size=config['hidden_size'],
    output_size=config['text_size'],
    n_layers=1,
    dropout=0.5,
)
seq2seq = Seq2Seq(encoder, decoder).to(get_device())
optimizer = optim.Adam(seq2seq.parameters(), lr=config['learning_rate'])
scheduler = optim.lr_scheduler.StepLR(
    optimizer,
    step_size=config['scheduler_step_size'],
    gamma=config['scheduler_gamma'],
)
criterion = nn.CrossEntropyLoss(ignore_index=vocab_config.stoi[Token.Padding.value]).to(get_device())
print(seq2seq)

[!] Instantiating model...


/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Seq2Seq(
  (encoder): EncoderRNN(
    (embedding): Embedding(68725, 128, padding_idx=1)
    (gru): GRU(128, 256, num_layers=2, dropout=0.5, bidirectional=True)
  )
  (decoder): DecoderRNN(
    (embedding): Embedding(68725, 128, padding_idx=1)
    (dropout): Dropout(p=0.5, inplace=True)
    (attention): BahdanauAttention(
      (attention): Linear(in_features=512, out_features=256, bias=True)
    )
    (gru): GRU(384, 256, dropout=0.5)
    (classifier): Linear(in_features=512, out_features=68725, bias=True)
  )
)


## Train model utils

In [29]:
def save_model(model, model_path: str, model_epoch) -> None:
    torch.save(model.cpu().state_dict(), model_path + f'_{model_epoch}.pt')
    torch.save(model.decoder.attention.v.cpu(), model_path + f'_att_param_{model_epoch}.pt')
    model.to(get_device())


def load_model(model, model_path: str, attention_param_path: str = None) -> None:
    if attention_param_path:
        model.load_state_dict(torch.load(model_path), strict=False)
        model.decoder.attention.v = nn.Parameter(torch.load(attention_param_path))
    else:
        model.load_state_dict(torch.load(model_path))


def get_text_summary_from_batch(batch):
    text = batch.text[0].to(get_device())
    summary = batch.summary[0].to(get_device())
    return text, summary


def show_rouge_and_attention_matrix(epoch, batch_id, text, summary):
    original_text = vocab_config.text_from_indices(text.transpose(0, 1)[0])
    target_summary = vocab_config.text_from_indices(summary.transpose(0, 1)[0])
    output_summary, attention = predict(
        vocab_config.text_from_indices(text.transpose(0, 1)[0]))
    logging.info(f'Original : {original_text}\n{"".join(["-" for i in range(80)])}'
                     f'Target : {target_summary}\n{"".join(["-" for i in range(80)])}'
                     f'Summary : {output_summary}\n{"".join(["-" for i in range(80)])}')
    scores = calculate_rouge(hypothesis=output_summary, reference=target_summary)
    if scores:
        for key, value in scores[0].items():
            logging.info(
                f'{key.upper()} [precision] : {np.round(value["p"] * 100, 2)} '
                f'| [recall] : {np.round(value["r"] * 100, 2)} '
                f'| [f-score] : {np.round(value["f"] * 100, 2)}',)
        draw_attention_matrix(
            attention=attention,
            original=original_text,
            summary=output_summary,
            config=config,
            epoch=epoch,
            batch_id=batch_id,
        )
    del original_text, target_summary, output_summary, attention, scores


def show_loss(batch_id, loss, train_iterator):
    print(
        f'[{batch_id} / {len(train_iterator)}] [loss: {loss}] '
        f'[lr: {optimizer.param_groups[0]["lr"]} ]')
    if AVAILABLE_GPU:
        torch.cuda.empty_cache()

## Train

In [30]:
def train(epoch, model, criterion, optimizer, scheduler, train_iter):
    grad_clip = config['grad_clip']
    text_size = config['text_size']
    model.train()
    total_loss = 0
    for batch_id, batch in tqdm(enumerate(train_iter), total=len(train_iter)):
        text, summary = get_text_summary_from_batch(batch)
        
        optimizer.zero_grad()
        output = model(text, summary)
        loss = criterion(
            output[1:].view(-1, text_size),
            summary[1:].contiguous().view(-1),
        )
        
        loss.backward()
        clip_grad_norm_(model.parameters(), grad_clip)
        optimizer.step()
        scheduler.step()
        total_loss += loss.data

        if batch_id % 100 == 0:
            show_loss(batch_id, loss.data, train_iter)

        if batch_id % 400 == 0:
            show_rouge_and_attention_matrix(epoch, batch_id, text, summary)

## Evaluate

In [31]:
def evaluate(model, criterion, val_iter):
    with torch.no_grad():
        text_size = config['text_size']
        total_loss = 0
        for batch_id, batch in enumerate(val_iter):
            text, summary = get_text_summary_from_batch(batch)
            
            output = model(text, summary, teacher_forcing_ratio=0.0)
            loss = criterion(
                output[1:].view(-1, text_size),
                summary[1:].contiguous().view(-1),
            )
            total_loss += loss.data
        return total_loss / len(val_iter)

## Predict

In [36]:
def predict(text, length_of_original_text=0.25):
    with torch.no_grad():
        sequence = vocab_config.indices_from_text(text).unsqueeze(0)
        sequence_length = sequence.size(1)
        encoder_outputs, encoder_hidden = encoder(sequence.transpose(0, 1))
        
        decoder_input = torch.LongTensor(
            [vocab_config.indices_from_text(Token.StartOfSentence.value)]).to(get_device())
        hidden = encoder_hidden[:decoder.n_layers]
        summary_words = [Token.StartOfSentence.value]
        max_summary_length = int(sequence_length * length_of_original_text)
        decoder_attentions = torch.zeros(max_summary_length, sequence_length)
        
        for idx in range(max_summary_length):
            output, hidden, decoder_attention = decoder(
                decoder_input, 
                hidden, 
                encoder_outputs,
            )
            decoder_attentions[idx, :decoder_attention.size(2)] += \
                decoder_attention.squeeze(0).squeeze(0).cpu().data
            top_v, top_i = output.data.topk(1)
            ni = top_i[0]
            if ni == vocab_config.indices_from_text(Token.EndOfSentence.value):
                break
            else:
                summary_words.append(vocab_config.text_from_indices(ni))
            
            decoder_input = torch.LongTensor([ni]).to(get_device())
        summary_words.append(Token.EndOfSentence.value)
        summary = " ".join(summary_words).lstrip()
        return summary, decoder_attentions

## Run

In [ ]:
best_loss = None
for epoch in tqdm(range(1, config['epochs'] + 1)):
    train(epoch, seq2seq, criterion, optimizer, scheduler, train_iter)
    valid_loss = seq2seq.evaluate(model, criterion, valid_iter)
    
    if not best_loss or valid_loss < best_loss:
        best_loss = valid_loss
        save_model(
            model=seq2seq, 
            model_path=os.path.join(config['model_output_path'], config['model_name']),
            model_epoch=epoch,
        )
        test_loss = seq2seq.evaluate(valid_iterator=test_iterator)
        logging.info(f'Test loss : {test_loss}')